In [3]:
import pandas as pd
train_data = pd.read_csv('./DataSets/train_modified.csv')
x = train_data[[x for x in train_data.columns if x not in ['Disbursed','ID']]]
y = train_data['Disbursed']
print(x.shape)
print(y.shape)
train_data.head()

(20000, 49)
(20000,)


,Disbursed,Existing_EMI,ID,Loan_Amount_Applied,Loan_Tenure_Applied,Monthly_Income,Var4,Var5,Age,EMI_Loan_Submitted_Missing,...,Var2_2,Var2_3,Var2_4,Var2_5,Var2_6,Mobile_Verified_0,Mobile_Verified_1,Source_0,Source_1,Source_2
0,0,0.0,ID000002C20,300000,5,20000,1,0,37,1,...,0,0,0,0,1,1,0,1,0,0
1,0,0.0,ID000004E40,200000,2,35000,3,13,30,0,...,0,0,0,0,1,0,1,1,0,0
2,0,0.0,ID000007H20,600000,4,22500,1,0,34,1,...,0,0,0,0,0,0,1,0,0,1
3,0,0.0,ID000008I30,1000000,5,35000,3,10,28,1,...,0,0,0,0,0,0,1,0,0,1
4,0,25000.0,ID000009J40,500000,2,100000,3,17,31,1,...,0,0,0,0,0,0,1,0,0,1


In [4]:
#用默认参数进行训练
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
rfm = RandomForestClassifier()
rfm.fit(x,y)
y_pred = rfm.predict(x)
metrics.accuracy_score(y_pred,y)

0.996

In [5]:
#用网格搜索确定基学习器数量
from sklearn.model_selection import GridSearchCV
gscv1 = GridSearchCV(RandomForestClassifier(),param_grid={'n_estimators':range(20,100,20)},cv=5)
gscv1.fit(x,y)
print(gscv1.best_score_,gscv1.best_params_)
gscv1.grid_scores_

0.98365 {'n_estimators': 60}


[mean: 0.98345, std: 0.00033, params: {'n_estimators': 20},
 mean: 0.98350, std: 0.00027, params: {'n_estimators': 40},
 mean: 0.98365, std: 0.00037, params: {'n_estimators': 60},
 mean: 0.98350, std: 0.00035, params: {'n_estimators': 80}]

In [7]:
#接着搜索最大深度和叶子节点最少样本数
gscv1 = GridSearchCV(RandomForestClassifier(n_estimators=60),param_grid={'max_depth':range(3,14,2), 'min_samples_split':range(50,201,20)},cv=5)
gscv1.fit(x,y)
print(gscv1.best_score_,gscv1.best_params_)
gscv1.grid_scores_

0.984 {'max_depth': 3, 'min_samples_split': 50}


[mean: 0.98400, std: 0.00000, params: {'max_depth': 3, 'min_samples_split': 50},
 mean: 0.98400, std: 0.00000, params: {'max_depth': 3, 'min_samples_split': 70},
 mean: 0.98400, std: 0.00000, params: {'max_depth': 3, 'min_samples_split': 90},
 mean: 0.98400, std: 0.00000, params: {'max_depth': 3, 'min_samples_split': 110},
 mean: 0.98400, std: 0.00000, params: {'max_depth': 3, 'min_samples_split': 130},
 mean: 0.98400, std: 0.00000, params: {'max_depth': 3, 'min_samples_split': 150},
 mean: 0.98400, std: 0.00000, params: {'max_depth': 3, 'min_samples_split': 170},
 mean: 0.98400, std: 0.00000, params: {'max_depth': 3, 'min_samples_split': 190},
 mean: 0.98400, std: 0.00000, params: {'max_depth': 5, 'min_samples_split': 50},
 mean: 0.98400, std: 0.00000, params: {'max_depth': 5, 'min_samples_split': 70},
 mean: 0.98400, std: 0.00000, params: {'max_depth': 5, 'min_samples_split': 90},
 mean: 0.98400, std: 0.00000, params: {'max_depth': 5, 'min_samples_split': 110},
 mean: 0.98400, std: 0

In [10]:
#搜索最大划分特征数
gscv1 = GridSearchCV(RandomForestClassifier(n_estimators=60,max_depth=13,min_samples_split=120,min_samples_leaf=20,oob_score=True),param_grid={'max_features':range(2,11,2)},cv=5)
gscv1.fit(x,y)
print(gscv1.best_score_,gscv1.best_params_)
gscv1.grid_scores_

0.984 {'max_features': 2}


[mean: 0.98400, std: 0.00000, params: {'max_features': 2},
 mean: 0.98400, std: 0.00000, params: {'max_features': 4},
 mean: 0.98400, std: 0.00000, params: {'max_features': 6},
 mean: 0.98400, std: 0.00000, params: {'max_features': 8},
 mean: 0.98400, std: 0.00000, params: {'max_features': 10}]

In [ ]:
#用选定的参数进行训练
rfm = RandomForestClassifier(n_estimators=60,max_depth=13,min_samples_split=120,min_samples_leaf=20,oob_score=True,max_features=7)
rfm.predict(x)
met